POR HACER
* funcion data augmentation y guardar las fotos creadas
* cargar archivos txt, pickle o hd5py


HECHOS
* crear modelo cambiando la ultima capa y congelar capas a partir de la ultima convolucional
* cargar pesos
* compilar
* muestras train, val y test

In [1]:
%run '../system/functions.ipynb' # import util functions

In [2]:
from time import time # execution time no the best accurate but the easiest sometimes does not work

In [3]:
# img_labels & img_list - completed path to file
PATH_TO_LABELS_FILE = "../data/output_dataset/img_labels"
PATH_TO_PATHS_FILE = "../data/output_dataset/img_list"

# output convnet files path + feed
PATH_TO_OUTPUT = "../data/output_convnet"
PATH_TO_DATA_AUGMENTATION = "../data/output_dataset/data_augmentation"

In [4]:
img_labels = []
img_list = []

'''
para eliminar el retorno de carro --> [:-1]
para encontrar el path correcto hay que transformar 
la ruta puesto que este fichero y desde el que se crea tienen rutas distintas
'''
# input_dataset/01_TUMOR/2642_CRC-Prim-HE-07_025.tif_Row_1351_Col_601.tif

with open(PATH_TO_LABELS_FILE,'r') as f_img_labels:
    for line in f_img_labels:
        img_labels.append(line[:-1]) 

with open(PATH_TO_PATHS_FILE,'r') as f_img_list:
    for line in f_img_list:
        img_list.append("../data/"+line[:-1]) # para eliminar el retorno de carro

In [5]:
import numpy as np

img_labels_arr = np.array(img_labels)
img_list_arr = np.array(img_list)

In [6]:
# proporciones elegidas 80-20 // entrenamiento y test
TRAIN = 0.8

# index selection
index_selection = np.arange(len(img_list)) # tamaño de la lista

np.random.shuffle(index_selection) # ahora en index_selection tenemos aleatorizado el vector

# ahora tenemos aleatorizados los vectores de la misma manera (pareados)

img_labels_arr_rand = img_labels_arr[index_selection]
img_list_arr_rand = img_list_arr[index_selection]

sep = int(TRAIN*len(img_list)) # [0:3999] -> 4000 images to train (validation included)

# print(sep)


# muestras de entrenamiento

labels_train = img_labels_arr_rand[:sep] 
img_list_train = img_list_arr_rand[:sep]

# muestras test

labels_test = img_labels_arr_rand[int(sep):] # 2999 +1 to 2999+1+1000
img_list_test = img_list_arr_rand[int(sep):]

In [ ]:
# # proporciones elegidas 60-20-20 // entrenamiento validacion y test
# TRAIN = 0.8

# # index selection
# index_selection = np.arange(len(img_list)) # tamaño de la lista

# np.random.shuffle(index_selection) # ahora en index_selection tenemos aleatorizado el vector

# # ahora tenemos aleatorizados los vectores de la misma manera (pareados)

# img_labels_arr_rand = img_labels_arr[index_selection]
# img_list_arr_rand = img_list_arr[index_selection]

# sep = int(TRAIN*len(img_list)) # [0:3999] -> 4000 images to train (validation included)

# # print(sep)


# # muestras de entrenamiento

# labels_train = img_labels_arr_rand[:sep] 
# img_list_train = img_list_arr_rand[:sep]

# # muestras test

# labels_test = img_labels_arr_rand[int(sep):] # 2999 +1 to 2999+1+1000
# img_list_test = img_list_arr_rand[int(sep):]

In [ ]:
# usando train_test_split de scikit learn
# http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html
# x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [7]:
print("tamaño muestra entrenamiento")
print(labels_train.shape[0])
print(img_list_train.shape[0])
print()

print("tamaño muestra test")
print(labels_test.shape[0])
print(img_list_test.shape[0])
print()

tamaño muestra entrenamiento
4000
4000

tamaño muestra test
1000
1000



comprobamos, que las etiquetas se correspondan

In [ ]:
# print(labels_train)
# print(img_list_train)

In [ ]:
# print(labels_test)
# print(img_list_test)

In [ ]:
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

copiamos las fotos en dos directorios para alimentar a la red neuronal

# BORRAR LAS FOTOS SI GENERAMOS OTRA VEZ LAS MUESTRAS

In [ ]:
# import shutil
# import os

# for img in img_list_train:
#     shutil.copy(img, 'train/')
    
# for img in img_list_test:
#     shutil.copy(img, 'test/')

In [8]:
n_categories = 8 # numero de categorias en las que clasificar las imagenes

In [9]:
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers.core import Flatten, Dense, Dropout
from keras.layers.convolutional import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.optimizers import SGD
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
import numpy as np

Using TensorFlow backend.


In [10]:
# comprobar si el modelo es vgg16 como el implementado en keras
# https://github.com/keras-team/keras/blob/master/keras/applications/vgg16.py # estructura implementada en keras
model = Sequential()
model.add(ZeroPadding2D((1,1),input_shape=(224,224,3)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(ZeroPadding2D((1,1)))
model.add(Conv2D(512, (3, 3), activation='relu'))
model.add(MaxPooling2D((2,2), strides=(2,2)))

model.add(Flatten())

#top layer of the VGG net
model.add(Dense(4096, activation='relu',name="fc1"))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='relu',name="fc2"))
model.add(Dropout(0.5))
model.add(Dense(1000, activation='softmax',name="last_layer"))

# model.summary() # lo oculto por ahora

In [11]:
model.load_weights("keras_weights/vgg16_weights_tf_dim_ordering_tf_kernels.h5")

In [ ]:
model.layers.pop()
model.outputs = [model.layers[-1].output]
model.layers[-1].outbound_nodes = []

# activacion softmax para calcular multiclasificacion
model.add(Dense(n_categories, activation='softmax',name="new_cats"))

In [ ]:
model.summary()

In [ ]:
# for i, layer in enumerate(model.layers):
#     print (i, layer.name, layer.output_shape)

In [ ]:
# congelamos todas salvo las fc1 fc2 y new_cats, en estas los pesos se actualizaran (entrenaran)
for layer in model.layers[:31]:
    layer.trainable = False

In [ ]:
for i, layer in enumerate(model.layers):
    if layer.trainable:
        print("layer {0:s} is trainable".format(layer.name))
    else:
        print("layer {0:s} is freezed".format(layer.name))

In [ ]:
# revisar estos hiperparametros --> leer sobre ellos y tomar decisiones
# sgd = SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)
# model.compile(optimizer=SGD(), loss='categorical_crossentropy', metrics=['accuracy'])

# compilamos este como el caso anterior para poder compararlos
model.compile(optimizer=SGD(), loss='categorical_crossentropy')

In [ ]:
# ahora ya hay que pasarle lista de fotografias a entrenar

In [ ]:
# https://keras.io/models/sequential/

scores = []

ti_bn_fit = time()

for i, index_path in enumerate(img_list_train):
    
    # img preparation
    img = image.load_img(index_path, target_size=(224, 224), interpolation='lanczos')
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    
    # model init
#     model.fit(x, labels_train[i], verbose=1, validation_split=0.2)
    model.fit(x, labels_train[i], verbose=1)

    # model eval
#     score = model.evaluate(x, labels_train[i], verbose=1)
    
#     scores.append(score[1] * 100)

tf_bn_fit = time()    

tt_bn_fit = tf_bn_fit - ti_bn_fit

In [ ]:
import time
print(time.strftime("%H:%M:%S", time.gmtime(tt_bn_fit)))

In [ ]:
#resultados

In [ ]:
# https://keras.io/models/sequential/
ti_bn_evaluate = time()

model.evaluate(img_list_test, labels_test,verbose=1)

tf_bn_evaluate = time()    

tt_bn_evaluate = tf_bn_evaluate - ti_bn_evaluate

In [ ]:
import time
print(time.strftime("%H:%M:%S", time.gmtime(tt_bn_evaluate)))

In [ ]:
#resultados

In [ ]:
# https://keras.io/models/sequential/
ti_bn_predict = time()

model.predict_classes(img_list_test,verbose=1)

tf_bn_predict = time()    

tt_bn_predict = tf_bn_predict - ti_bn_predictt

In [ ]:
import time
print(time.strftime("%H:%M:%S", time.gmtime(tt_bn_predict)))

In [ ]:
#resultados